# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [79]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import os
import pprint
import seaborn as sns 
import geopandas as gpd
import cartopy.crs as ccrs
import geoviews as gv # noqa

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key


In [80]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,23.09,100,75,1.03,TO,1670261728
1,1,ushuaia,-54.8000,-68.3000,11.79,46,40,11.83,AR,1670261555
2,2,sao joao do paraiso,-15.3136,-42.0144,21.73,78,100,2.80,BR,1670261729
3,3,richards bay,-28.7830,32.0377,21.26,91,100,8.20,ZA,1670261729
4,4,bengkulu,-3.8004,102.2655,23.66,93,100,1.67,ID,1670261730


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [94]:
url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-0.07071648508463113,51.50848194136378,10000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=500&apiKey={geoapify_key}"
url

'https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-0.07071648508463113,51.50848194136378,10000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=500&apiKey=addd1a5d64e741f986dd85210a080359'

In [95]:
city_url = requests.get(url).json()
city_url

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'The Tower Hotel',
    'street': "St. Katharine's Way",
    'neighbourhood': 'East Smithfield',
    'suburb': 'Wapping',
    'city': 'London',
    'county': 'Greater London',
    'state': 'England',
    'postcode': 'E1W 1LD',
    'country': 'United Kingdom',
    'country_code': 'gb',
    'lon': -0.0731951454698233,
    'lat': 51.50660995,
    'formatted': "The Tower Hotel, St. Katharine's Way, London E1W 1LD, United Kingdom",
    'address_line1': 'The Tower Hotel',
    'address_line2': "St. Katharine's Way, London E1W 1LD, United Kingdom",
    'categories': ['accommodation',
     'accommodation.hotel',
     'building',
     'building.accommodation',
     'wheelchair',
     'wheelchair.limited'],
    'details': ['details',
     'details.building',
     'details.facilities',
     'details.wiki_and_media'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contrib

In [108]:
# # Configure the map

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
#     geo =True,
#     size = "Humidity"
#     color = "City"
)

# Display the map plot_1
map_plot_1



:Points   [Lng,Lat]

In [107]:
city_data_cleaned = city_data_df.dropna()
city_data_cleaned.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,23.09,100,75,1.03,TO,1670261728
1,1,ushuaia,-54.8000,-68.3000,11.79,46,40,11.83,AR,1670261555
2,2,sao joao do paraiso,-15.3136,-42.0144,21.73,78,100,2.80,BR,1670261729
3,3,richards bay,-28.7830,32.0377,21.26,91,100,8.20,ZA,1670261729
4,4,bengkulu,-3.8004,102.2655,23.66,93,100,1.67,ID,1670261730
5,5,miri,4.4148,114.0089,26.20,94,40,2.06,MY,1670261730
6,6,butaritari,3.0707,172.7902,27.13,80,82,7.80,KI,1670261731
7,7,tasiilaq,65.6145,-37.6368,-0.91,63,79,3.23,GL,1670261732
8,8,busselton,-33.6500,115.3333,12.91,89,51,5.39,AU,1670261732
9,9,ribeira grande,38.5167,-28.7000,17.22,67,20,6.71,PT,1670261733


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
#cloudiness is zero:
cloudiness_df = city_data_cleaned[city_data_cleaned["Cloudiness"] == 0]

# A max temperature lower than 27 degrees but higher than 21:
temp_df = cloudiness_df[(cloudiness_df["Max Temp"] > 21) & (cloudiness_df["Max Temp"] < 27)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 4.5]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,moranbah,-22.0016,148.0466,21.72,72,0,3.47,AU,1670261751
59,59,graciano sanchez,23.9167,-99.2833,23.53,38,0,2.43,MX,1670261755
111,111,salalah,17.0151,54.0924,24.31,66,0,0.34,OM,1670261690
159,159,doha,25.2867,51.5333,25.94,71,0,2.57,QA,1670261551
275,275,hilo,19.7297,-155.0900,24.60,84,0,3.09,US,1670261775
387,387,kahului,20.8947,-156.4700,24.36,82,0,2.24,US,1670261902
448,448,upington,-28.4478,21.2561,26.21,17,0,2.06,ZA,1670261929
525,525,jiwani,25.0500,61.7417,24.90,56,0,3.88,PK,1670261966


In [60]:
#another way to confirm the ideal weather condition 

ideal_weather_df = city_data_cleaned.loc[(city_data_cleaned["Max Temp"] < 27)
                                    & (city_data_cleaned["Max Temp"] > 21) 
                                    & (city_data_cleaned["Wind Speed"] < 4.5) 
                                    & (city_data_cleaned["Cloudiness"] == 0),:].dropna()
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,moranbah,-22.0016,148.0466,21.72,72,0,3.47,AU,1670261751
59,59,graciano sanchez,23.9167,-99.2833,23.53,38,0,2.43,MX,1670261755
111,111,salalah,17.0151,54.0924,24.31,66,0,0.34,OM,1670261690
159,159,doha,25.2867,51.5333,25.94,71,0,2.57,QA,1670261551
275,275,hilo,19.7297,-155.0900,24.60,84,0,3.09,US,1670261775
387,387,kahului,20.8947,-156.4700,24.36,82,0,2.24,US,1670261902
448,448,upington,-28.4478,21.2561,26.21,17,0,2.06,ZA,1670261929
525,525,jiwani,25.0500,61.7417,24.90,56,0,3.88,PK,1670261966


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_cleaned[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vaini,TO,-21.2000,-175.2000,100,
1,ushuaia,AR,-54.8000,-68.3000,46,
2,sao joao do paraiso,BR,-15.3136,-42.0144,78,
3,richards bay,ZA,-28.7830,32.0377,91,
4,bengkulu,ID,-3.8004,102.2655,93,
5,miri,MY,4.4148,114.0089,94,
6,butaritari,KI,3.0707,172.7902,80,
7,tasiilaq,GL,65.6145,-37.6368,63,
8,busselton,AU,-33.6500,115.3333,89,
9,ribeira grande,PT,38.5167,-28.7000,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [100]:
# Set parameters to search for a hotel
hotel_df = wind_df
# &filter=circle:-0.07071648508463113,51.50848194136378,10000&bias=proximity:-0.07071648508463113,51.50848194136378&limit=500
        
latitude = 51.50848194136378
longitude = -0.07071648508463113
   
limit = 500
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key
}

# radius = 10000
# params = {
#     "types": "hotel",
#     "key": geoapify_key
# }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"{lat},{lng}"
    params["bias"] = f"{lat}, {lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
moranbah - nearest hotel: No hotel found
graciano sanchez - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
doha - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
upington - nearest hotel: No hotel found
jiwani - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
50,50,moranbah,-22.0016,148.0466,21.72,72,0,3.47,AU,1670261751,No hotel found
59,59,graciano sanchez,23.9167,-99.2833,23.53,38,0,2.43,MX,1670261755,No hotel found
111,111,salalah,17.0151,54.0924,24.31,66,0,0.34,OM,1670261690,No hotel found
159,159,doha,25.2867,51.5333,25.94,71,0,2.57,QA,1670261551,No hotel found
275,275,hilo,19.7297,-155.0900,24.60,84,0,3.09,US,1670261775,No hotel found
387,387,kahului,20.8947,-156.4700,24.36,82,0,2.24,US,1670261902,No hotel found
448,448,upington,-28.4478,21.2561,26.21,17,0,2.06,ZA,1670261929,No hotel found
525,525,jiwani,25.0500,61.7417,24.90,56,0,3.88,PK,1670261966,No hotel found


In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [110]:
%%capture --no-display

map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
#     geo =True,
#     size = "Humidity"
#     color = "City"
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot_1
map_plot_2



:Points   [Lng,Lat]   (Country)